In [101]:
# -*- coding: utf-8 -*-
"""
TransformerPredictTrainModel.ipynb

Entrena el modelo BigramLanguageModel utilizando un dataset
(leído desde un CSV) y aplica una tokenización subword (BPE)
mediante SentencePiece. Al finalizar, guarda los pesos del modelo 
y varios parámetros necesarios para la inferencia posterior.
"""

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import re
import sentencepiece as spm
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter
# -------------------------------------------------------------------------------
# HIPERPARÁMETROS
# -------------------------------------------------------------------------------
batch_size = 64
block_size = 256
n_embd = 384
n_head = 6
n_layer = 4
dropout = 0.2
max_iters = 4000
eval_interval = 200
lr = 3e-4
weight_decay = 1e-2
grad_accum_steps = 2  # Gradient Accumulation
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [102]:
# -------------------------------------------------------------------------------
# NORMALIZACIÓN DE TEXTO (ÁRABE)
# -------------------------------------------------------------------------------
def normalize_arabic(text):
    """
    Elimina diacríticos y unifica ciertos caracteres.
    """
    # Quita diacríticos.
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    # (Opcional) Unificar alif: 
    # text = re.sub(r'[إأآا]', 'ا', text)
    return text

In [103]:
# -------------------------------------------------------------------------------
# LECTURA DE CSV Y GENERACIÓN DE ARCHIVO DE TEXTO
# -------------------------------------------------------------------------------
df_poetry = pd.read_csv('Arabic_Poetry_Dataset.csv')

# Si solo te interesa la columna 'poem_text':
raw_text = "\n".join(df_poetry['poem_text'].astype(str).tolist())

# Luego normalizas como quieras
text_normalized = normalize_arabic(raw_text)

print("Tamaño del texto (caracteres) después de normalizar:", len(text_normalized))


Tamaño del texto (caracteres) después de normalizar: 39570083


In [104]:
# Guardar el corpus en un archivo .txt
with open("corpus_ar.txt", "w", encoding="utf-8") as f:
    f.write(text_normalized)

In [105]:
# -------------------------------------------------------------------------------
# ENTRENAR SENTENCEPIECE (BPE)
# -------------------------------------------------------------------------------
spm.SentencePieceTrainer.Train(
    '--input=corpus_ar.txt '
    '--model_prefix=arabic_spm '
    '--vocab_size=16000 '     # se proba para obtener mejores resultado (ej. 4k, 8k, 16k)
    '--model_type=bpe '      # Byte Pair Encoding
    '--character_coverage=1.0 '
    '--user_defined_symbols=<pad>,<s>,</s> '
    '--unk_piece=<unk>'
)

In [106]:
# -------------------------------------------------------------------------------
# CARGAR EL MODELO DE TOKENIZACIÓN
# -------------------------------------------------------------------------------
sp = spm.SentencePieceProcessor()
sp.load("arabic_spm.model")

# funciones de encode/decode
def encode(s: str):
    """Convierte un string en una lista de IDs de subword."""
    return sp.encode(s, out_type=int)

def decode(ids: list):
    """Convierte una lista de IDs en el texto (string)."""
    return sp.decode(ids)

# El tamaño de vocab = número de subwords en el modelo
vocab_size = sp.get_piece_size()
print("Tamaño del vocabulario (subwords) =", vocab_size)

Tamaño del vocabulario (subwords) = 16000


In [107]:
# -------------------------------------------------------------------------------
# CREAR EL TENSOR DE DATOS (DATA) A PARTIR DEL TEXTO
# -------------------------------------------------------------------------------
# Convierte el texto normalizado a IDs de subwords
encoded_ids = encode(text_normalized)
# Monta el tensor PyTorch
data = torch.tensor(encoded_ids, dtype=torch.long)
print("Longitud total de tokens (subwords) =", len(data))

Longitud total de tokens (subwords) = 10744625


In [108]:
# -------------------------------------------------------------------------------
# Dividir en train y val (95/10)
# -------------------------------------------------------------------------------
n = int(0.99 * len(data))
train_data = data[:n]
val_data = data[n:]

In [109]:
print("Tamaño del dataset de entrenamiento:", len(train_data))
print("Tamaño del dataset de validación:", len(val_data))

Tamaño del dataset de entrenamiento: 10637178
Tamaño del dataset de validación: 107447


In [ ]:
# -------------------------------------------------------------------------------
# FUNCIONES PARA OBTENER BATCHES Y ESTIMAR PÉRDIDA
# -------------------------------------------------------------------------------
def get_batch(split: str):
    """
    Devuelve un batch (x, y) para el conjunto train o val.
    
    Args:
        split (str): 'train' o 'val', para indicar de qué conjunto sacar el batch.
        
    Returns:
        x (tensor): Secuencias de entrada (batch_size, block_size).
        y (tensor): Objetivos esperados (batch_size, block_size).
    """
    data_source = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_source) - block_size, (batch_size,))
    
    x = torch.stack([data_source[i : i + block_size] for i in ix])
    y = torch.stack([data_source[i + 1 : i + block_size + 1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    """
    Evalúa la pérdida promedio en entrenamiento (train)
    y validación (val) usando 'eval_iters' iteraciones.
    """
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
def build_rope_sin_cos(head_dim, block_size, base=10000):
    """
    Construye tensores de seno y coseno para posiciones [0, block_size-1],
    asumiendo que 'head_dim' es la dimensión de cada cabeza (debe ser par).

    Args:
        head_dim (int): Dimensión de cada cabeza (por ejemplo, 64).
        block_size (int): Longitud máxima de la secuencia.
        base (int): Base para la escala de frecuencia (usualmente 10000).

    Returns:
        sin (Tensor): Tensor de senos de forma (1, block_size, head_dim//2).
        cos (Tensor): Tensor de cosenos de forma (1, block_size, head_dim//2).
    """
    half_dim = head_dim // 2
    # Calcular frecuencias para cada componente de la mitad
    freq_seq = torch.arange(0, half_dim, dtype=torch.float32)
    freq_seq = base ** (2 * freq_seq / (head_dim * 2))  # Escala según la dimensión

    positions = torch.arange(block_size, dtype=torch.float32).unsqueeze(1)  # (block_size, 1)
    div_term = positions / freq_seq.unsqueeze(0)  # (block_size, half_dim)
    
    sin = torch.sin(div_term).unsqueeze(0)  # (1, block_size, half_dim)
    cos = torch.cos(div_term).unsqueeze(0)  # (1, block_size, half_dim)
    return sin, cos

def apply_rope(q, k, rope_sin, rope_cos):
    """
    Aplica Rotary Positional Embeddings a los tensores de consulta y clave
    de un único head.

    Args:
        q (Tensor): Tensor de consulta de forma (B, T, head_dim).
        k (Tensor): Tensor de clave de forma (B, T, head_dim).
        rope_sin (Tensor): Tensores de seno, de forma (1, block_size, head_dim//2).
        rope_cos (Tensor): Tensores de coseno, de forma (1, block_size, head_dim//2).

    Returns:
        q_final, k_final (Tensors): Tensores rotados, de forma (B, T, head_dim).
    """
    B, T, C = q.shape  # C = head_dim
    half_dim = C // 2

    # Dividir q y k en dos mitades
    q_left, q_right = q[..., :half_dim], q[..., half_dim:]
    k_left, k_right = k[..., :half_dim], k[..., half_dim:]
    
    # Seleccionar las posiciones (hasta T) y expandir para tener (B, T, half_dim)
    sin_t = rope_sin[:, :T, :].expand(B, T, half_dim)
    cos_t = rope_cos[:, :T, :].expand(B, T, half_dim)
    
    # Aplicar la rotación: 
    # q_rot = q_left * cos - q_right * sin
    # q_pass = q_left * sin + q_right * cos
    q_rot = q_left * cos_t - q_right * sin_t
    q_pass = q_left * sin_t + q_right * cos_t
    q_final = torch.cat((q_rot, q_pass), dim=-1)
    
    k_rot = k_left * cos_t - k_right * sin_t
    k_pass = k_left * sin_t + k_right * cos_t
    k_final = torch.cat((k_rot, k_pass), dim=-1)
    
    return q_final, k_final

In [ ]:
# -------------------------------------------------------------------------------
# ENCABEZADO DEL MODELO
# -------------------------------------------------------------------------------
class Head(nn.Module):
    """
    Cabeza de self-attention con máscara causal y aplicación de RoPE.
    
    Esta implementación es para un único head; se espera que la dimensión
    de entrada 'n_embd' en este módulo corresponda a 'head_dim' (por ejemplo, 64).
    """
    def __init__(self, n_embd, head_size, block_size, dropout):
        """
        Args:
            n_embd (int): Dimensión de entrada (debe ser igual a head_size para este head).
            head_size (int): Tamaño de cada cabeza (por ejemplo, 64).
            block_size (int): Longitud máxima de la secuencia.
            dropout (float): Tasa de dropout.
        """
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        
        # Máscara causal: para que cada token no vea tokens futuros.
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, rope_sin, rope_cos):
        """
        Args:
            x (Tensor): Entrada de forma (B, T, head_dim), donde head_dim = head_size.
            rope_sin, rope_cos (Tensors): Tensores de RoPE con forma (1, block_size, head_dim//2).
        
        Returns:
            out (Tensor): Salida de la cabeza, de forma (B, T, head_size).
        """
        B, T, C = x.shape
        q = self.query(x)  # (B, T, head_size)
        k = self.key(x)
        v = self.value(x)
        
        # Aplicar RoPE a q y k
        q_rot, k_rot = apply_rope(q, k, rope_sin, rope_cos)
        
        # Calcular atención escalada
        scale = C ** -0.5
        wei = q_rot @ k_rot.transpose(-2, -1) * scale  # (B, T, T)
        
        # Aplicar máscara causal
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        # Combinar la atención con los valores
        out = wei @ v  # (B, T, head_size)
        return out

# -------------------------------------------------
# BLOQUES DEL TRANSFORMER (con FeedForward usando GELU)
# -------------------------------------------------
class MultiHeadAttention(nn.Module):
    """
    Ejemplo de MultiHeadAttention (sin details de RoPE, 
    pero podrías integrarlo aquí).
    """
    def __init__(self, n_embd, n_heads, block_size, dropout):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = n_embd // n_heads
        self.block_size = block_size

        self.wq = nn.Linear(n_embd, n_embd, bias=False)
        self.wk = nn.Linear(n_embd, n_embd, bias=False)
        self.wv = nn.Linear(n_embd, n_embd, bias=False)
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

        # Máscara causal
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape

        # Proyecciones
        q_full = self.wq(x)  # (B, T, n_embd)
        k_full = self.wk(x)
        v_full = self.wv(x)

        # Dividir heads
        q = q_full.view(B, T, self.n_heads, self.head_dim).permute(0,2,1,3)
        k = k_full.view(B, T, self.n_heads, self.head_dim).permute(0,2,1,3)
        v = v_full.view(B, T, self.n_heads, self.head_dim).permute(0,2,1,3)

        # Atención
        wei = q @ k.transpose(-2, -1) * (self.head_dim**-0.5)
        mask = self.tril[:T, :T]
        wei = wei.masked_fill(mask==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = wei @ v  # (B, n_heads, T, head_dim)
        # Reordenar
        out = out.permute(0,2,1,3).contiguous()
        out = out.view(B, T, self.n_heads*self.head_dim)
        # Proyección final
        out = self.proj(out)
        out = self.dropout(out)
        return out


class FeedForward(nn.Module):
    """
    MLP tras la atención, usando GELU en lugar de ReLU.
    """
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.GELU(),           # Cambiamos ReLU -> GELU
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """
    Bloque Transformer con:
    - MultiHeadAttention
    - FeedForward (con GELU)
    - Residual connections
    """
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHeadAttention(n_embd, n_head, block_size, dropout)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ffwd = FeedForward(n_embd, dropout)

    def forward(self, x):
        # Atención
        x_ln = self.ln1(x)
        attn_out = self.sa(x_ln)
        x = x + attn_out
        # FF
        x_ln2 = self.ln2(x)
        ff_out = self.ffwd(x_ln2)
        x = x + ff_out
        return x


class BigramLanguageModel(nn.Module):
    """
    Modelo de lenguaje con:
    - Embeddings
    - Bloques (Multi-Head + FeedForward con GELU)
    - Funciones de generación con top-k / top-p / temperatura
    """
    def __init__(self, vocab_size, block_size, n_embd, n_head, n_layer, dropout=0.2):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_embd = n_embd
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # Pila de bloques
        self.blocks = nn.ModuleList([
            Block(n_embd, n_head, block_size, dropout) 
            for _ in range(n_layer)
        ])

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        if T > self.block_size:
            raise ValueError("Secuencia excede block_size")

        x = self.token_embedding_table(idx)  # (B, T, n_embd)

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(
        self, 
        idx, 
        max_new_tokens: int, 
        temperature: float=1.0, 
        top_k: int=None, 
        top_p: float=None
    ):
        """
        Genera tokens con top-k / top-p sampling y temperatura.
        idx: (B, T) secuencia de tokens
        max_new_tokens: cuántos tokens generar
        temperature: escalado de logits
        top_k: mantiene sólo los k más probables
        top_p: nucleus sampling, mantiene sólo tokens cumul>= top_p
        """
        for _ in range(max_new_tokens):
            # Cortar contexto
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            # Tomar logits del último token
            logits = logits[:, -1, :]  # (B, vocab_size)

            # Ajustar por temperatura
            if temperature != 1.0:
                logits = logits / temperature

            # Convertir a probabilidades
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)

            # top-k
            if (top_k is not None) and (top_k > 0):
                # Coger los k mejores
                values, indices = torch.topk(probs, top_k, dim=-1)
                probs_topk = torch.zeros_like(probs).scatter_(-1, indices, values)
                probs_topk = probs_topk / probs_topk.sum(dim=-1, keepdim=True)
                probs = probs_topk

            # top-p (nucleus sampling)
            if (top_p is not None) and (0.0 < top_p < 1.0):
                sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
                cumulative = torch.cumsum(sorted_probs, dim=-1)
                # máscara booleana: qué tokens entran en el top_p
                cutoff = (cumulative > top_p).float()
                # shift para que el primero que supere top_p se incluya
                cutoff = torch.roll(cutoff, 1, dims=-1)
                cutoff[..., 0] = 0
                # poner 0 en los tokens que exceden top_p
                sorted_probs[cutoff.bool()] = 0.0
                # renormalizar
                sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
                # re-esparcir
                probs_top_p = torch.zeros_like(probs).scatter_(-1, sorted_indices, sorted_probs)
                probs = probs_top_p

            # muestrear
            idx_next = torch.multinomial(probs, num_samples=1)
            # concatenar
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
# -------------------------------------------------
# ENTRENAMIENTO AVANZADO
# -------------------------------------------------

def get_batch(train_data, val_data, batch_size, block_size, split='train', device='cuda'):
    """
    Devuelve un batch (x, y) para el conjunto train o val.
    """
    data_source = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_source) - block_size, (batch_size,))
    x = torch.stack([data_source[i : i + block_size] for i in ix])
    y = torch.stack([data_source[i + 1 : i + block_size + 1] for i in ix])
    return x.to(device), y.to(device)

def train_model():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # HIPERPARÁMETROS
    batch_size = 64
    block_size = 256
    n_embd = 512 
    n_head = 8
    n_layer = 4
    dropout = 0.2
    max_iters = 7000
    eval_interval = 200
    lr = 3e-4
    weight_decay = 1e-2
    grad_accum_steps = 2  # Gradient Accumulation

    # Instanciar el modelo
    model = BigramLanguageModel(
        vocab_size=vocab_size,
        block_size=block_size,
        n_embd=n_embd,
        n_head=n_head,
        n_layer=n_layer,
        dropout=dropout
    ).to(device)

    # Define el número de pasos para el warmup
    warmup_steps = 500  # Por ejemplo, los primeros 500 pasos

    # max_iters es el total de iteraciones de entrenamiento (después del warmup)
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            # Warmup lineal: lr aumenta linealmente de 0 a 1
            return float(current_step) / float(max(1, warmup_steps))
        # Después del warmup, se aplica cosine decay
        progress = float(current_step - warmup_steps) / float(max(1, max_iters - warmup_steps))
        return 0.5 * (1.0 + math.cos(math.pi * progress))

    # Suponiendo que ya definiste el optimizador:
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = LambdaLR(optimizer, lr_lambda)


    # Ejemplo de Cosine Scheduler + Warmup
    # (PyTorch no trae un "WarmupCosine" nativo, 
    #  lo implementamos con un simple wrapper)
    scheduler = CosineAnnealingLR(optimizer, T_max=max_iters, eta_min=1e-5)

    def estimate_loss():
        # evalúa la pérdida en train/val
        out = {}
        model.eval()
        with torch.no_grad():
            for split in ['train','val']:
                losses = []
                for _ in range(5):  # 5 mini-batches de eval
                    xb, yb = get_batch(train_data, val_data, batch_size, block_size, split, device)
                    _, loss_eval = model(xb, yb)
                    losses.append(loss_eval.item())
                out[split] = sum(losses)/len(losses)
        model.train()
        return out

    model.train()
    running_loss = 0.0
    
    
    writer = SummaryWriter(log_dir="runs/experiment1")
    for iter in range(max_iters):
        # Gradient Accumulation: 
        # Acumulamos grad en 'grad_accum_steps' sub-batches
        loss_local = 0.0
        for _ in range(grad_accum_steps):
            xb, yb = get_batch(train_data, val_data, batch_size, block_size, 'train', device)
            _, loss = model(xb, yb)
            loss.backward()
            loss_local += loss.item()
        # Actualizamos (un step cada grad_accum_steps sub-batches)
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss_local

        # LR Scheduler
        scheduler.step()

        # Imprimir cada eval_interval
        if iter % eval_interval == 0:
            losses_eval = estimate_loss()
            writer.add_scalar("Loss/train", losses_eval["train"], iter)
            writer.add_scalar("Loss/val", losses_eval["val"], iter)
            print(f"Step {iter}: train_loss={losses_eval['train']:.4f}, val_loss={losses_eval['val']:.4f}")


        # (Opcional) Ajustar dropout dinámicamente (no estándar, 
        #  se hace manual, p.ej. reducirlo gradualmente):
        # if iter < 500:
        #     new_drop = 0.3
        # else:
        #     new_drop = 0.1
        # for m in model.modules():
        #     if isinstance(m, nn.Dropout):
        #         m.p = new_drop

    print("Entrenamiento finalizado")
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'vocab_size': vocab_size,
        'block_size': block_size,
        'n_embd': n_embd,
        'n_head': n_head,
        'n_layer': n_layer
    }
    torch.save(checkpoint, 'bigram_model_checkpoint.pt')
    print("Checkpoint guardado en 'bigram_model_checkpoint.pt'")
    return model

trained_model = train_model()
